# Documentation Code Example from:
- https://spark.apache.org/docs/latest/ml-classification-regression.html#linear-regression

In [1]:
import findspark
findspark.init('/home/kajili/spark-2.4.5-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
training = spark.read.format("libsvm").load("./sample_linear_regression_data.txt")

In [6]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [7]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [8]:
lrModel = lr.fit(training)

In [9]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [10]:
lrModel.intercept

0.14228558260358093

In [11]:
training_summary = lrModel.summary

In [12]:
training_summary.r2

0.027839179518600154

In [13]:
training_summary.rootMeanSquaredError

10.16309157133015

In [14]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [15]:
split_data_object = all_data.randomSplit([0.7,0.3])
split_data_object

[DataFrame[label: double, features: vector],
 DataFrame[label: double, features: vector]]

In [18]:
#instead do this:
train_data,test_data = all_data.randomSplit([0.7,0.3])
train_data

DataFrame[label: double, features: vector]

In [19]:
test_data

DataFrame[label: double, features: vector]

In [23]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                343|
|   mean|0.23439297628832237|
| stddev| 10.731601406880289|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [22]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                158|
|   mean| 0.3057247408404779|
| stddev|  9.389521574681957|
|    min|-28.046018037776633|
|    max|  23.52945433069272|
+-------+-------------------+



In [24]:
correct_model = lr.fit(train_data)

In [25]:
test_results = correct_model.evaluate(test_data)

In [26]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -25.92113203722698|
|-26.865540827031225|
| -19.68357557493384|
|-15.927982212530555|
|-15.453796807094573|
|-18.805362367324413|
|-15.317070536283781|
|-17.080322163485434|
|-14.233525375617402|
|-16.426126493329473|
| -15.25295188690791|
|-17.688364723178594|
| -8.432654153480222|
|-10.986191767708645|
|-15.169163025757857|
|-14.305381388025666|
|-12.893063694353987|
|-15.419745601255245|
| -7.729319415523667|
|-12.574165081849614|
+-------------------+
only showing top 20 rows



In [27]:
test_results.rootMeanSquaredError

9.509429501592862

In [28]:
unlabeled_data = test_data.select('features')

In [29]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [30]:
predictions = correct_model.transform(unlabeled_data)

In [31]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...|-2.1248860005496546|
|(10,[0,1,2,3,4,5,...| 3.9157148908351522|
|(10,[0,1,2,3,4,5,...| 0.8376531020352562|
|(10,[0,1,2,3,4,5,...| -1.566218144352788|
|(10,[0,1,2,3,4,5,...|-1.8729239255813743|
|(10,[0,1,2,3,4,5,...| 1.7399627414483967|
|(10,[0,1,2,3,4,5,...|-0.7685885047377092|
|(10,[0,1,2,3,4,5,...| 1.3568065504368658|
|(10,[0,1,2,3,4,5,...|-1.1423323476948954|
|(10,[0,1,2,3,4,5,...|   1.36964351878704|
|(10,[0,1,2,3,4,5,...| 0.4901936339767834|
|(10,[0,1,2,3,4,5,...| 3.3593862141031514|
|(10,[0,1,2,3,4,5,...| -4.340572845770975|
|(10,[0,1,2,3,4,5,...|-1.5723840211475444|
|(10,[0,1,2,3,4,5,...|  2.668389240402802|
|(10,[0,1,2,3,4,5,...|  2.107284823364254|
|(10,[0,1,2,3,4,5,...|  0.798712415818728|
|(10,[0,1,2,3,4,5,...| 3.5926726048626736|
|(10,[0,1,2,3,4,5,...|-3.9112302623651583|
|(10,[0,1,2,3,4,5,...|  1.628245424066682|
+----------